In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torch.utils.data import DataLoader, TensorDataset

In [2]:
work_dir = os.getcwd()#.replace("GitHub/RocketPill/ipynb-notebooks", "Data/")
work_dir

'/Users/rckyi/Documents/GitHub/RocketPill/ipynb-notebooks'

In [3]:
data_dir = os.getcwd().replace("GitHub/RocketPill/ipynb-notebooks", "Data/")
data_dir

'/Users/rckyi/Documents/Data/'

In [4]:
thrust_curves_data_path = data_dir + "thrust_curves_normalized.csv"
thrust_curves_data_path

'/Users/rckyi/Documents/Data/thrust_curves_normalized.csv'

In [5]:
models_path = os.getcwd().replace("GitHub/RocketPill/ipynb-notebooks", "Deep_Learning_Models/") 
models_path

'/Users/rckyi/Documents/Deep_Learning_Models/'

In [6]:
def fetch_impulse_classes():
   
    # URL of the Wikipedia page for motor classes
    url = "https://en.wikipedia.org/wiki/Model_rocket_motor_classification"
    
    # Send a request to the webpage
    response = requests.get(url)
    # print(f'response json {response.json()}')
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all tables
    tables = soup.find_all('table', {'class': 'wikitable'})
    
    # Look for the table that contains the header "Total impulse (N·s)"
    target_table = None
    for table in tables:
        # print(f'table text {table.}')
        if 'impulse' in table.text:
            target_table = table
            break
    # print(target_table)
    # Parse the table into a DataFrame
    df = pd.read_html(str(tables))[0]
    
    # Display the resulting DataFrame
    # print(df['Class  (Base 26)'])
    ls = df['Class  (Base 26)'].tolist()
    
    return [x for x in ls if len(x) < 10]

In [7]:
# 1. ThrustCurve.org API access
BASE_URL = "https://www.thrustcurve.org/api/v1"

def search_motors(impulse_class="F"):
    response = requests.get(f"{BASE_URL}/search.json", params={"impulseClass": impulse_class})
    response.raise_for_status()
    data = response.json()
    # print(f'motor results {data.get("results", [])[0]}')
    return data.get("results", [])  # not "motors"

def download_thrust_curve(motor_id):
    response = requests.get(f"{BASE_URL}/download.json", params={"motorId": motor_id, "data": "samples"})
    response.raise_for_status()
    data = response.json()
    if "samples" in str(data["results"]):
        # print(f'samples found')
        return data["results"][0]["samples"]
    return []

In [8]:
def normalize_curve(curve):
    # print(f'curve time: {[x.get("time") for x in curve]}')
    if not curve:
        print(f'No curve')
        return None
    curve = np.array(curve)
    time = np.asarray([x.get("time") for x in curve]) # curve[:, 0]
    thrust = np.asarray([x.get("thrust") for x in curve]) # curve[:, 1]
    time = (time - time.min()) / (time.max() - time.min())
    thrust = thrust / thrust.max()
    return pd.DataFrame({"time": time, "thrust": thrust})

In [9]:
def collect_data(impulse_class, max_motors):
    motors = search_motors(impulse_class)
    # print(f'motor: {motors[0]}')
    print(f'{len(motors)} Motors found: {[x["motorId"] for x in motors]}')
    dataset = []
    for motor in tqdm(motors[:max_motors]):
        # print(f'motor {motor}')
        motor_id = motor["motorId"]
        motor_name = motor.get("designation", "unknown")
        samples = download_thrust_curve(motor_id)
        norm_df = normalize_curve(samples)
        # print(f'norm df {norm_df}')
        if norm_df is not None:
            norm_df["motorId"] = motor_id       
            norm_df["motor_name"] = motor_name
            norm_df["impulse_class"]=impulse_class
            dataset.append(norm_df)
            # print(f'norm_df {norm_df["motor_name"]}')
    final_df = pd.concat(dataset, ignore_index=True)
    thrust_curves_data_path = data_dir + impulse_class +'_' + "thrust_curves_normalized.csv"
    final_df.to_csv(thrust_curves_data_path, index=False)
    print("Saved dataset to thrust_curves_normalized.csv")

In [10]:
if __name__ == "__main__":
    classes = fetch_impulse_classes()
    print(f'classes: {classes}')
    for c in classes:
        print(f'IMPULSE CLASS {c} ===============================================')
        try:
            collect_data(c, max_motors=50)
        except:
            print(f'Data not found for {c} impulse class')
            pass
    print(f'Done fetching data')
        

classes: ['Micro', '1/4A', '1/2A', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ']
IMPULSE CLASS Micro ===============================================
Data not found for Micro impulse class
IMPULSE CLASS 1/4A ===============================================
0 Motors found: []


0it [00:00, ?it/s]

Data not found for 1/4A impulse class
IMPULSE CLASS 1/2A ===============================================


0 Motors found: []


0it [00:00, ?it/s]

...

Saved dataset to thrust_curves_normalized.csv
IMPULSE CLASS O ===============================================
13 Motors found: ['5f4294d200023100000003ee', '5f4294d2000231000000046c', '5f4294d20002310000000401', '5f4294d2000231000000042e', '5f4294d200023100000001c4', '5f4294d200023100000003ae', '5f4294d200023100000001c3', '5f4294d20002310000000216', '5f4294d200023100000003c4', '5f4294d2000231000000042f', '5f4294d2000231000000028c', '656e92d97f1a4b00027f1588', '5f4294d20002310000000215']


 77%|██████████████████████████████████████████████████████████████████████████████▍                       | 10/13 [00:01<00:00,  5.44it/s]

No curve


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  4.98it/s]


Saved dataset to thrust_curves_normalized.csv
IMPULSE CLASS P ===============================================
Data not found for P impulse class
IMPULSE CLASS Q ===============================================
Data not found for Q impulse class
IMPULSE CLASS R ===============================================
Data not found for R impulse class
IMPULSE CLASS S ===============================================
Data not found for S impulse class
IMPULSE CLASS T ===============================================
Data not found for T impulse class
IMPULSE CLASS U ===============================================
Data not found for U impulse class
IMPULSE CLASS V ===============================================
Data not found for V impulse class
IMPULSE CLASS W ===============================================
Data not found for W impulse class
IMPULSE CLASS X ===============================================
Data not found for X impulse class
IMPULSE CLASS Y ===============================================